In [13]:
import pandas as pd
import numpy as np

df = pd.read_csv('claims_train.csv')
df['ClaimFrequency'] = df['ClaimNb'] / df['Exposure']
df=df[df['Exposure'] >= 0.01]
cap = df['ClaimFrequency'].quantile(0.995)
df['ClaimFrequency'] = df['ClaimFrequency'].clip(upper=cap)
df.head()

num_cols = ['VehPower', 'VehAge', 'DrivAge', 'BonusMalus', 'Density', 'Exposure']
cat_cols = ['Area']

df_dummies = pd.get_dummies(df[cat_cols], drop_first=False)

X_train = pd.concat([df[num_cols], df_dummies], axis=1).values
y_train = df['ClaimFrequency'].values

class decisionTree:
    num_cols = ['VehPower', 'VehAge', 'DrivAge', 'BonusMalus', 'Density', 'Exposure']
cat_cols = ['Area']

df_dummies = pd.get_dummies(df[cat_cols], drop_first=False)

X_train = pd.concat([df[num_cols], df_dummies], axis=1).values
y_train = df['ClaimFrequency'].values

class decisionTree:
    def __init__(self):
        self.tree = None

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.tree = {
            "is_leaf": True,
            "value": float(y.mean())
        }

    def predict_row(self, row, node):
        return node["value"]

    def predict(self, X):
        X = np.asarray(X)
        return np.array([self.predict_row(row, self.tree) for row in X])

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    return np.sqrt(((y_true - y_pred) ** 2).mean())

tree = decisionTree()
tree.fit(X_train, y_train)
y_train_pred = tree.predict(X_train)

print(tree.tree)
print("Training RMSE:", rmse(y_train, y_train_pred))


{'is_leaf': True, 'value': 0.13825320390506804}
Training RMSE: 0.83604324672754
